# Speed comparison between numpy, cupy and numba for a simple agent-based model

author: damien.challet@centralesupelec.fr (2022)  

In [6]:
import numpy as np
import cupy as cp    
import numba

In [8]:
bnds = ((0.0001, 0.1),
        (0.001,0.3),
        (0.001,50),
        (0.001,0.3), 
        (0.0001,0.05),
        (0.0001,0.1),
        (-0.1,0.1)
       )

params0=np.array(bnds)[:,0]*1.5
params0[-1]=0
params_names=["alpha","beta","gamma","kappa","sigma_N","lambda","g"]
params0

array([0.00015, 0.0015 , 0.0015 , 0.0015 , 0.00015, 0.00015, 0.     ])

## cupy

In [9]:
def abm(params,Nruns,NIT):
    alpha=params[0]
    beta=params[1]
    gamma=params[2]
    kappa=params[3]
    sigma_N=params[4]
    lamda=params[5]
    g=params[6]

    r=cp.zeros((Nruns,NIT))
    eps=cp.random.normal(0,sigma_N,(Nruns,NIT))
    m=g*cp.ones(Nruns ) # E(m)=g
    v=cp.zeros(Nruns)
    p=cp.zeros(Nruns)
    for t in range(NIT):
        r[:,t]=kappa*(v-p)+beta*cp.tanh(gamma*m)+eps[:,t]+g
        p=p+r[:,t]
        m=(1-alpha)*m+alpha*r[:,t]
        v=(1-lamda)*v+lamda*p
    return r

In [10]:
%%timeit

abm(params0,20000,300)

105 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numpy

In [16]:
def abm(params,Nruns,NIT):
    alpha=params[0]
    beta=params[1]
    gamma=params[2]
    kappa=params[3]
    sigma_N=params[4]
    lamda=params[5]
    g=params[6]

    r=np.zeros((Nruns,NIT))
    eps=np.random.normal(0,sigma_N,(Nruns,NIT))
    m=g*np.ones(Nruns ) # E(m)=g
    v=np.zeros(Nruns)
    p=np.zeros(Nruns)
    for t in range(NIT):
        r[:,t]=kappa*(v-p)+beta*np.tanh(gamma*m)+eps[:,t]+g
        p=p+r[:,t]
        m=(1-alpha)*m+alpha*r[:,t]
        v=(1-lamda)*v+lamda*p
    return r

In [17]:
%%timeit

abm(params0,20000,300)

395 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numba

In [18]:
import numba
from numba import njit
from numba.typed import List
@njit
def abm(params,Nruns,NIT):
    alpha=params[0]
    beta=params[1]
    gamma=params[2]
    kappa=params[3]
    sigma_N=params[4]
    lamda=params[5]
    g=params[6]

    r=np.zeros((Nruns,NIT))
    eps=np.random.normal(0,sigma_N,(Nruns,NIT))
    m=g*np.ones(Nruns ) # E(m)=g
    v=np.zeros(Nruns)
    p=np.zeros(Nruns)
    for t in range(NIT):
        r[:,t]=kappa*(v-p)+beta*np.tanh(gamma*m)+eps[:,t]+g
        p=p+r[:,t]
        m=(1-alpha)*m+alpha*r[:,t]
        v=(1-lamda)*v+lamda*p
    return r

In [19]:
params0=List(params0)

In [20]:
%%timeit

abm(params0,20000,300)

474 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
